<a href="https://colab.research.google.com/github/Carlageo9/Practica_3_Metricas/blob/main/Practica_3_Metricas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tópicos de Indistria I - Práctica 3 Métricas

**Nombre:** Carla Georgina Sánchez Arreguín

**e-mail:** carla.sanchez2472@alumnos.udg.mx


*   Actividad 1: Path-length - (BM1 vs BM2 vs CRW)
*   Actividad 2: Mean Squared Displacement - (BM vs CRW)
*   Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada)
*   Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada)




MODULES

In [1]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

FUNCTIONS

In [3]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    #Declaramos un vector que va desde la pos_a hasta la pos_b
    vec_ab = np.array([pos_b[0] - pos_a[0], pos_b[1] - pos_a[1]])
    #Se calcula la longitud de ese vector
    norm_ab = np.linalg.norm(vec_ab)

    #Declaramos un vector que va desde la pos_b hasta la pos_c
    vec_bc = np.array([pos_c[0] - pos_b[0], pos_c[1] - pos_b[1]])
    #Se calcula la longitud de ese vector
    norm_bc = np.linalg.norm(vec_bc)

    #Con la función dot de NumPy multiplicamos los vectores
    dot_p = np.dot(vec_ab, vec_bc)

    #Se calcula el coseno del ángulo entre los vectores
    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = dot_p / (norm_ab * norm_bc + np.finfo(float).eps)

    # Angle orientation
    #Calculamos el producto de cruz o producto vectorial
    cross_p = np.cross(vec_ab, vec_bc)
    #Determinamos la orientación del ángulo
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    #La función arccos de NumPy devuelve el arcocoseno de los elementos de la estructura de entrada
    #Calculamos el ángulo en radianes utilizando arccos y se multiplica por la orientación
    theta = np.arccos(np.around(cos_theta,4)) * orient

    return theta

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [5]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [6]:
# Define your function to compute path length for given trajectory
###########################################################
### Brownian Motion (BM)
###########################################################
def bm_2d(n_steps = 1000, speed = 6, s_pos = [0, 0]):
  """
  Arguments:
    n_steps:
    speed:
    s_pos:
  Returns:
    BM_2d_df:
  """

  #Inicializamos el vector de velocidad
  velocity = Vec2d(speed, 0)

  #Declaramos los DataFrame para guardar la trayectoria
  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  #con un for repetimos ciertas condiciones según el número de pasos definido
  for i in range(n_steps-1):
    #Generamos un ángulo de giro aleatorio
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    #Actualizamos la dirección en la que se generará el movimiento
    velocity = velocity.rotated(turn_angle)
    #Calculamos la nueva posición sumando la actual con recién generada
    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
    #Actualizamos la nueva posición
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df




In [20]:
# Define your function to compute path length for given trajectory
###########################################################
### Correlated Random Walks (CRW)
###########################################################
def crw_2d(n_steps = 1000, speed = 6, s_pos = [0, 0]):
  """
  Arguments:
    n_steps:
    speed:
    s_pos:
  Returns:
    CRW_2d_df:
  """

  #Inicializamos el vector de velocidad
  velocity = Vec2d(speed, 0)

  #Declaramos los DataFrame para guardar la trayectoria
  CRW_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  distribucion = wrapcauchy(0.4, 0, scale=1)
  giros = distribucion.rvs(size=n_steps)

  #con un for repetimos ciertas condiciones según el número de pasos definido
  for i in range(n_steps-1):
    #Generamos un ángulo de giro aleatorio
    turn_angle = giros[i]
    #Actualizamos la dirección en la que se generará el movimiento
    velocity = velocity.rotated(turn_angle)
    #Calculamos la nueva posición sumando la actual con recién generada
    temp_df = pd.DataFrame([{'x_pos': CRW_2d_df.x_pos[i]+velocity.x, 'y_pos': CRW_2d_df.y_pos[i]+velocity.y}])
    #Actualizamos la nueva posición
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  return CRW_2d_df




In [12]:
###########################################################
### Función para calcular distancia entre 2 puntos
###########################################################
def distancia(x1, y1, x2, y2):
    """
    Calcula la distancia euclidiana entre dos puntos en un espacio bidimensional.

    Args:
        x1 (float): Coordenada x del primer punto.
        y1 (float): Coordenada y del primer punto.
        x2 (float): Coordenada x del segundo punto.
        y2 (float): Coordenada y del segundo punto.

    Returns:
        float: Distancia euclidiana entre los dos puntos.
    """
    #Con la función math.sqrt calculamos la raíz cuadrada, y con ** elevamos a la segunda potencia
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

In [23]:
# Generamos trayectorias
BM_2d_df_3 = bm_2d(1000, 3, [0, 0])
BM_2d_df_6 = bm_2d(1000, 6, [0, 0])
#CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')
CRW_2d_df_9 = crw_2d(1000, 6, [0, 0])


#Calculamos distancia y path length para BM3
dis_BM_3 = np.array([distancia(BM_2d_df_3['x_pos'].iloc[i-1], BM_2d_df_3['y_pos'].iloc[i-1], BM_2d_df_3['x_pos'].iloc[i], BM_2d_df_3['y_pos'].iloc[i]) for i in range(1, BM_2d_df_3.shape[0])])
pl_BM_3 = np.cumsum(dis_BM_3)

#Calculamos distancia y path length para BM6
dis_BM_6 = np.array([distancia(BM_2d_df_6['x_pos'].iloc[i-1], BM_2d_df_6['y_pos'].iloc[i-1], BM_2d_df_6['x_pos'].iloc[i], BM_2d_df_6['y_pos'].iloc[i]) for i in range(1, BM_2d_df_6.shape[0])])
pl_BM_6 = np.cumsum(dis_BM_6)

#Calculamos distancia y path length para CRW6
dis_CRW_6 = np.array([distancia(CRW_2d_df_9['x_pos'].iloc[i-1], CRW_2d_df_9['y_pos'].iloc[i-1], CRW_2d_df_9['x_pos'].iloc[i], CRW_2d_df_9['y_pos'].iloc[i]) for i in range(1, CRW_2d_df_9.shape[0])])
pl_CRW_6 = np.cumsum(dis_CRW_6)

#Creamos un DataFrame para almacenar las métricas
df_metrics = pd.DataFrame({
    'Step': np.arange(2, BM_2d_df_3.shape[0] + 1),
    'Path Length BM 3': pl_BM_3,
    'Path Length BM 6': pl_BM_6,
    'Path Length CRW 6': pl_CRW_6
})

# Inicializamos figura
fig_path_length = go.Figure()

# Añadimos trayectoria BM3
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(pl_BM_3)) + 1,
    y=pl_BM_3,
    name='Path Length BM 3',
    showlegend=True))

# Añadimos trayectoria BM6
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(pl_BM_6)) + 1,
    y=pl_BM_6, line=dict(width=8),
    name='Path Length BM 6',
    showlegend=True))

# Añadimos trayectoria CRW6
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(pl_CRW_6)) + 1,
    y=pl_CRW_6,
    name='Path Length CRW 6',
    showlegend=True))

fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (4 pts)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [10]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [11]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()